# Sparse matrices

somewhat you (probably) will need to work with the first competition efficiently.


The scipy.sparse module provides data structures for 2D sparse matrices. There are [several](http://www.scipy-lectures.org/advanced/scipy_sparse/storage_schemes.html) available sparse matrix types, we'll use two of them: 

1. csc_matrix: Compressed Sparse Column 
2. csr_matrix: Compressed Sparse Row 

Actually, we already used them.

## Some copy-paste from notebook 1.2

In [1]:
import numpy
from scipy import sparse
from sklearn.metrics import roc_auc_score

In [2]:
import codecs
# python kung fu. No need to understand this code
with codecs.open('./data/1-restaurant-train.csv') as f:
    labels, reviews = zip(*[line.split('\t') for line in f.readlines()])
with codecs.open('./data/1-restaurant-test.csv') as f:
    kaggle_test_reviews = f.readlines()

In [3]:
import pandas
from IPython.display import FileLink
# supplementary function. Again, just use it
def create_solution(predictions, filename='1-restaurant-predictions.csv'):
    result = pandas.DataFrame({'Id': numpy.arange(len(predictions)), 'Solution': predictions})
    result.to_csv('data/{}'.format(filename), index=False)
    return FileLink('data/{}'.format(filename))

In [4]:
answers = numpy.array(labels, dtype=int) >= 4

## Using CountVectorizer

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
# take 100000 most frequent words
vectorizer = CountVectorizer(max_features=100000)
vectorizer.fit(reviews)

counts = vectorizer.transform(reviews) #.toarray()
kaggle_test_counts = vectorizer.transform(kaggle_test_reviews) #.toarray()

In [6]:
counts

<82065x92353 sparse matrix of type '<type 'numpy.int64'>'
	with 9493523 stored elements in Compressed Sparse Row format>

### output of vectorizer is sparse ...

In [7]:
# take first 100 lines, convert to numpy
counts[:100].toarray() 
# but doing above operation with whole dataset is a very bad idea: ~10 ** 10 elements

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
n_elements = counts.shape[0] * counts.shape[1]
n_nonzero_elements = counts.getnnz()

# real amount of elements is ~1000 times less
print n_elements / float(n_nonzero_elements)

798.328391367


In [9]:
counts

<82065x92353 sparse matrix of type '<type 'numpy.int64'>'
	with 9493523 stored elements in Compressed Sparse Row format>

In [10]:
counts.T

<92353x82065 sparse matrix of type '<type 'numpy.int64'>'
	with 9493523 stored elements in Compressed Sparse Column format>

In [11]:
# we can see that transposing is instant, because it simply changed rows and columns.
%timeit counts.T

10000 loops, best of 3: 41.7 µs per loop


# Some operations

In [12]:
counts.sum()

14623064

In [13]:
counts.sum(axis=0)

matrix([[2234,  349,    1, ...,    1,    1,    1]])

In [14]:
# converted to numpy.array, then converted from matrix to a single row
numpy.array(counts.sum(axis=0))[0]

array([2234,  349,    1, ...,    1,    1,    1])

##  Adding new features

In [15]:
counts.max(axis=1) # max times one word was repeated in the review

<82065x1 sparse matrix of type '<type 'numpy.int64'>'
	with 82065 stored elements in COOrdinate format>

In [16]:
def add_new_features(sparse_counts):
    # added two new features to the dataset: number of words in the sentence and 'max repetitions'
    # horizontal stacking of matrices. Important - second and third are also matrices, but with one column
    return sparse.hstack([sparse_counts, sparse_counts.sum(axis=1), sparse_counts.max(axis=1)])

In [17]:
new_counts = add_new_features(counts)
# the same operations should be done on kaggle test (to use the same model)
new_kaggle_counts = add_new_features(kaggle_test_counts)

# Also let's add count of different characters

why not?

In [18]:
def add_character_count(counts, reviews, characters):
    new_features = numpy.zeros(shape=[len(reviews), len(characters)])
    for i, character in enumerate(characters):
        for j in range(len(reviews)):
            new_features[j, i] = reviews[j].lower().count(character)
    print new_features
    return sparse.hstack([counts, new_features])

In [19]:
characters = [')', '(', '!', 'like', ' ']
new_counts = add_character_count(new_counts, reviews, characters=characters)
new_kaggle_counts = add_character_count(new_kaggle_counts, kaggle_test_reviews, characters=characters)

[[   0.    0.    7.    1.  106.]
 [   0.    0.    1.    0.  158.]
 [   0.    0.    0.    0.  142.]
 ..., 
 [   1.    0.    0.    0.  213.]
 [   0.    0.    0.    0.  115.]
 [   2.    2.    0.    0.  104.]]
[[   1.    1.    1.    0.   96.]
 [   0.    0.    3.    0.  154.]
 [   0.    0.    0.    0.  149.]
 ..., 
 [   0.    0.    0.    2.  317.]
 [   1.    1.    0.    2.  105.]
 [   0.    0.    0.    0.  189.]]


# Train / test splitting

In [20]:
from sklearn.cross_validation import train_test_split

train_counts, test_counts, train_answers, test_answers = \
    train_test_split(new_counts, answers, train_size=10000, random_state=42)

/Users/antares/.virtualenvs/rep/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
train_counts # works correctly with sparse types too!

<10000x92360 sparse matrix of type '<type 'numpy.float64'>'
	with 1209508 stored elements in Compressed Sparse Row format>

## Training models

many (but not all!) models in scikit-learn work efficiently with sparse data

In [22]:
# logistic regression was explained in the lectures.
from sklearn.linear_model import LogisticRegression

# C is an inverse regularization, smaller C -> stronger regularization
logreg_clf = LogisticRegression(C=1.)
logreg_clf.fit(train_counts, train_answers)

# important: logistic regression is a CLASSIFIER, so we use .predict_proba!
print 'train: ', roc_auc_score(train_answers, logreg_clf.predict_proba(train_counts)[:, 1])
print 'test:  ', roc_auc_score(test_answers, logreg_clf.predict_proba(test_counts)[:, 1])

create_solution(logreg_clf.predict_proba(new_kaggle_counts)[:, 1], filename='1-restaurant-predictions-logreg.csv')

train:  0.999995552399
test:   0.879890597236


/Users/antares/Yandex.Disk.localized/2017-01-Imperial/data/1-restaurant-predictions-logreg.csv

In [23]:
# elastic net is a linear regression with both L1 and L2 regularizations, it's more general compared to Ridge
# sidenote: logistic regression also has L1 regularization
from sklearn.linear_model import ElasticNet, Ridge

# ridge_reg = ElasticNet(l1_ratio=0.5, alpha=0.1, max_iter=5)
ridge_reg = Ridge()
ridge_reg.fit(train_counts, train_answers)

# Ridge (and ElasticNet) is a regression method, so we use .predict method
print 'train: ', roc_auc_score(train_answers, ridge_reg.predict(train_counts))
print 'test:  ', roc_auc_score(test_answers, ridge_reg.predict(test_counts))

create_solution(ridge_reg.predict(new_kaggle_counts), filename='1-restaurant-predictions-ridge.csv')

train:  0.903638617114
test:   0.87573470998


/Users/antares/Yandex.Disk.localized/2017-01-Imperial/data/1-restaurant-predictions-ridge.csv

# References

- [scipy documentation on sparse matrices](https://docs.scipy.org/doc/scipy-0.18.1/reference/sparse.html)
    - more verbose reference can be found in [scipy lectures](http://www.scipy-lectures.org/advanced/scipy_sparse/index.html)
- ask in the gitter chat if you need something specific